<a href="https://colab.research.google.com/github/TasnimJumara/TableAssignmentFirst/blob/master/Experiment2_a(2ndDataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from customDataset import DigitFindingDataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd

In [ ]:
batch_size=1000
num_iteration=25000
input_dimension=28*28
num_hiddennodes=10
output_dimension=10
learning_rate=.001
hidden_layers=3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:

train_dataset = (r "/content/drive/MyDrive/Assignment/Dataset_2/verification_train.csv" train=True, transform=transforms.ToTensor())

test_dataset = (r "/content/drive/MyDrive/Assignment/Dataset_2/verification_test.csv" train=True, transform=transforms.ToTensor())


num_epochs = num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)   
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)  

In [ ]:
class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hiddennodes):
        super().__init__()
        self.linear_1 = nn.Linear(input_size, num_hiddennodes)
        self.tanh_1 = nn.Tanh()

        #self.linear_2 = nn.Linear(num_hiddennodes, num_hiddennodes)
        #self.tanh_2 = nn.Tanh()
  
        self.linear_3 = nn.Linear(num_hiddennodes, num_hiddennodes)
        self.tanh_3 = nn.Tanh()

        self.linear_out = nn.Linear(num_hiddennodes, num_classes)

    def forward(self, x):
       
        out = self.linear_1(x)
        out = self.tanh_1(out)
        
        #out  = self.linear_2(out)
       # out = self.tanh_2(out)

        out = self.linear_3(out)
        out = self.tanh_3(out)
        
        probas  = self.linear_out(out)
        return probas

model = DeepNeuralNetworkModel(input_size = input_dimension,
                               num_classes = output_dimension,
                               num_hiddennodes = num_hiddennodes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iteration_loss = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images) 

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)

                total += labels.size(0)


                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            iteration_loss.append(loss.item())
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

print (iteration_loss)
plt.plot(iteration_loss)
plt.ylabel('Cross Entropy Loss')
plt.xlabel('Iteration (in every 500)')
plt.show()